# Script for preprocessing tweets for bi-LSTM training #

**Author:** [Andrew Larkin](https://www.linkedin.com/in/andrew-larkin-525ba3b5/) <br>
**Affiliation:** [Oregon State University, College of Public Health and Human Sciences](https://health.oregonstate.edu/) <br>
**Date Created:** October 14th, 2018 <br>

**Summary** <br>
This script contains functions used to preprocess tweets and corresponding metadata as features for deep learning model input preprocessing includes:

### TODO:
[ ] add link to Stanford word vector website

- tokenization
- partitioning multiple word hashtags (e.g. #lastchildinthewoods) into multiple words and adjusting hashtag indicators and sentence length metadata
- calculating word frequency
- encoding words to 300 length word vectors created by Stanford 
- creating trainable, randomly initizlied word encodings for unknown and pad token words
- replace uknown words with an UNKNOWN tag
- fill the end sentences with empty tag so all tweet vectors are the same length


**Note**: text was already partially processed at the time of record insertion into the SQL database.  Input text has already had emojis replaced with word descriptions, hashtag and emoji indicators created, and hyperlinks removed.  

### Setup ###

** Import libraries, define model and dataset parameters, and set filepaths **

In [124]:
# libraries
import zipfile
import numpy as np
#import tensorflow as tf
import pandas as ps
import re, string
from IPython.display import Image
from IPython.core.display import HTML 

In [17]:
# filepaths
parentFolder =  "C:/Users/larkinan/Desktop/DBTraining/"

In [94]:
# hyperparameters and constants for the deep learning model.  
# While only the word_vec_dim is needed for this script, 
# the entire dictionary is copied here to facilitate ease of search and comparison.
modelParams = {
    'word_vec_dim':300, # dimension of each word vector
    'mini_batch_size':256, 
    'learning_rate':0.01,
    'momentum':0.9,
    'num_outcomes':1, # whether testing for just 1 or multiple outcomes,
    'hidden_layer size':32,
    'num_epochs':5000,
    'hidden_layer_activation':'tanh'
}

twitterCSVParams = {
    'filepath':parentFolder + "trainingDatav8.csv",
    'classifyFilepath':parentFolder + "allScores__Oct14_18.csv",
    'text':'clean_text',  # whether to use cleaned or raw (i.e. with emojis, hashtags, etc) twitter text
    'nature_ind':'nature',
    'safety_ind':'safety',
    'beauty_ind':'beauty',
    'exercise_ind':'exercise',
    'social_ind':'social',
    'stress_ind':'stress',
    'air_ind':'air',
    'hashtag_ind':'hash_ind',
    'emoticon_ind':'emot_ind' 
}

wordVecParams = {
    'zipFilename':parentFolder + "glove.840B.300d.zip",
    'txtFilename':"glove.840B.300d.txt"
}

unknownWordSubs = { # identified using the 'countUnknownWords' function
    "'04":'4',
    "'10":'10',
    "'12":'12',
    "'14":'14',
    "'15":'15',
    "'16":'16',
    "'17":'17',
    "'18":'18',
    "'72":'72',
    "'93":'93',
    "'96":'96',
    "'cause":'because',
    "'ll":'will',
    "'m":'am',
    "'re":'are',
    "'s":'is',
    "'ve":'have',
    "'Ve":'have',
    "'em":'them'
}

### convert binary variable to two binary variables ###
Classification in Tensorflow requires a unique vector for each classification level.  This function converts a single binary classification vector into two binary classification vectors, one vector for positive classification (value = 1 in the original input vector) and one vector for negative classification (value = 0 in the original input vector 

**TODO: <br> **
[  ] consider optimizing by converting for loop into vector operation.  But, it runs very fast even with a loop.  Maybe python is unrolling the loop?

**Inputs:** <br>
- **inputLabels ** (n x 1 numpy matrix) - Values are either 1 for positive classification and 0 for negative classification.  
- **debug** (boolean) - whether or not to print debug statements for testing/validation

**Outputs:** <br>
- **twoClassLabels** (n x 2 numpy matrix).  First column values are 1 for positive classification, 0 otherwise.  Second column values are 1 for negative classification, 0 otherwise.

In [3]:
def expandOneClassLables(inputLabels,debug):
    twoClassLabels = np.zeros((len(inputLabels),2))
    index = 0
    for label in inputLabels:
        if(int(label) == 0):
            twoClassLabels[index][1] = 1
        else:
            twoClassLabels[index][0] = 1
        index+=1
    print(twoClassLabels[0:5])
    print(inputLabels[0:5])
    return(twoClassLabels)

### transforming class labels for Tensorflow compatibility ###
Tensorflow requires one vector for each potential class. This function transforms a binary vector of class labels into two numpy arrays, one for positive classification and one for negative classification.  Finally, arrays for all 7 outcomes are combined to create a single nx14 numpy matrix. 

**Inputs:** <br>
- **inputLabels** (int arrays) - a list of int arrays, with one array (i.e. class labels) for each outcome of interest <br>
- **debug** (boolean) - whether ot not to print debug statements

**Outputs** <br>
- **LabelMatrix** (numpy matrix) - n x14 numpy matrix of expanded outcomes, with outcomes listed in the same column order as inputs.


![Alt text](https://raw.githubusercontent.com/larkinandy/GreenTweet_MultivariateBiLSTM/master/DataPreprocessing/images/methods_images.jpg)
>> **Figure 1.** Diagram depicting the input and output of expandAllClassLabels function. 

In [134]:
def expandAllClassLabels(inputLabels,debug):
    # TODO: complete function.  Use expandOneClassLabels as a subfunction
    pass

### create mapping dictionaries for all words in the twitter dataset ###
The mapping dictionaries are imporrtant for converting words into vectors and vice versa in later functions. Note that dictionaries include punctuation and  

**Inputs: ** <br>
- **inputSents ** (list) - twitter sentences containing all candidate words to map to

**Outputs: ** <br>
- ** index2word_map ** (dict) - dictionary with index as the key and word as the value
- ** word2index_map ** (dict) - dictionary with words as the key and index as the value

** TODO: ** consider vectorizing, but it runs very fast even with nested loops (< 1s for 100,000 sentences)

In [43]:
def generateWordMap(inputSents):
    word2index_map = {}
    index = 0
    for sent in inputSents:
        reducedPunc = string.punctuation.replace("'", " ")
        for word in sent:
            if(len(word)>0 and word[0] == "'"):                
                word = word[1:]
            if(len(word)>0 and word[-1] == "'"):
                word = word[:-1]
            if(len(word)>0 and word not in word2index_map):
                    word2index_map[word] = index
                    index+=1
    index2word_map = {word:index for word, index in word2index_map.items()}
    return index2word_map, word2index_map

In [5]:
# load class scores and twitter data from csv files 
def loadCSV(inputCSVFile,debug=False,inDelim=',',inQuoteChar='"'):
    rawData = ps.read_csv(inputCSVFile,encoding='utf-8',delimiter = inDelim,quotechar = inQuoteChar)
    if(debug):
        print(rawData.head())
    assert(rawData.isnull().any().any() == False)
    rawData.tweet_id = rawData.tweet_id.astype(str)
    return(rawData)

In [6]:
# extract csv filepaths from param dictionary 
def getTwitterCSVFilepaths(twitterDict):
    CSVFilepath = twitterCSVParams['filepath']
    classifyFilepath = twitterCSVParams['classifyFilepath']
    return CSVFilepath, classifyFilepath

In [7]:
# load twitter data features 
# Get the names of features and class labels from the twitter params dictionary
def getTwitterCSVParams(twitterDict):
    CSVFilepath = twitterCSVParams['filepath']
    classifyFilepath = twitterCSVParams['classifyFilepath']
    textVar = twitterCSVParams['text']
    hashtagVar = twitterCSVParams['hashtag_ind']
    emoticonVar = twitterCSVParams['emoticon_ind']
    natureInd = twitterCSVParams['nature_ind']
    safetyInd = twitterCSVParams['safety_ind']
    beautyInd = twitterCSVParams['beauty_ind']
    exerciseInd = twitterCSVParams['exercise_ind']
    socialInd = twitterCSVParams['social_ind']
    stressInd = twitterCSVParams['stress_ind']
    return (
        [CSVFilepath,classifyFilepath, textVar, hashtagVar, emoticonVar],
        [natureInd,safetyInd,beautyInd,exerciseInd,socialInd,stressInd]
    )

### load class labels and twitter data from csv files and merge ###

**Inputs** <br>
- **twitterCSVParams** (dict) - contains column names and filepaths.  See Setup section above for more details
- **debug** (boolean) - whether to print debug statements in subfunctions

**Outputs** <br>
- **tweetWithScores ** (dict) - contains class labels and twitter data merged into single dictionary.  Dictionary params are documented in the setup section above

**TODO** <br>
[ ] - combine csv files and simplify the data loading process

In [37]:
def loadWordList(twitterCSVParams,debug=False):
    TwitterFilepath, classifyFilepath = getTwitterCSVFilepaths(twitterCSVParams)
    classScores = loadCSV(classifyFilepath,debug)
    tweetData = loadCSV(TwitterFilepath,debug,inQuoteChar='|')
    tweetWithScores = tweetData.merge(classScores,  how='inner', left_on='tweet_id', right_on = 'tweet_id')
    return tweetWithScores

### load the word vectors ###
Read word vectors from file and load into memory.  Note that this file is large and compressed.  Recommend reading from an SSD drive and a computer with at least 16GB of memory (run on intel 900p storage with with 32GB RAM).

**Inputs:** <br>
- **vectorFilepath** (string) - entire filepath to zip file containing word vectors 

**Outputs:** <br>
- **embeddingWeights** (dict) - dictionary, where words are the keys and vectors are the values 

In [9]:
def loadWordVec(wordVecParams):
    embedding_weights = {}
    test = 0
    with zipfile.ZipFile(wordVecParams['zipFilename']) as z:
        with z.open(wordVecParams['txtFilename']) as f:
            for line in f:
                vals = line.split()
                word = str(vals[0].decode("utf-8"))
                coefs = np.asarray(vals[1:], dtype = 'float32')
                coefs/=np.linalg.norm(coefs)
                embedding_weights[word] = coefs
    return embedding_weights

### find the longest word at the end of a string of characters.  Return -1 if no word exists###
this function is for partitioning multiple words in tweets, compacted together without a space (e.g. lastchildinthewoods)

**Inputs:** <br>
- **wordSpace** (string) - string that potentially contains multiple words concatenated without spaces
- **inDict** (dictionary) - the word search space, i.e. list of all words we consider to be 'valid'
- **debug** (boolean) - whether to print debug statements

**Outputs:** <br>
- integer where string can be partitioned to subset words from wordSpace.  -1 if no such partition exists 

In [22]:
def searchForFirstWord(wordSpace,inDict,debug=False):
    lastWordLen = len(wordSpace)
    while(lastWordLen > 0):
        candidateWord = wordSpace[0:lastWordLen]
        if(candidateWord in inDict):
            endInd = lastWordLen
            if(debug):
                print(
                    "found long word in wordspace: \n word: %s \n index: %i" 
                    % (candidateWord,endInd)
                )
            return endInd
        lastWordLen -=1
    return -1

### for a single string of an unknown word, partition the string into smaller words ###
This function also extends the hashtag and emoticon indicator vectors corresponding to the increase in word length

**Inputs:** <br>
- **inWord** (string) - word to partition
- **inDict** (dict) - contains all acceptable or 'legal' words
- **isHashtag** (int array) - binary vector indicating whether each word in a sentence is from  a hashtag. Update to reflect new words
- **isEmot** (int array) - binary vector indicating whether each word in a sentence if from an emoticon.  Update to reflect new words
- **debug** (boolean) - whether to print debug statements 

**Outputs:** <br>
- **wordVec** (string array) - list of words, in sequential order of original sentence
- **outHashtag** (int array) - binary hashtag vector, updated to reflect any added words
- **outEmot** (int array) - binary emot vector, updated to reflect any added words

In [24]:
def checkOneCompactedWord(inWord,inDict,isHashtag,isEmot,debug=False):
    wordVec, outHashtag, outEmot = ([] for i in range(3))
    origWord = inWord
    partitionIndex = searchForFirstWord(inWord,inDict,debug)
    while(partitionIndex >0):
        outHashtag.append(isHashtag)
        outEmot.append(isEmot)
        wordVec.append(inWord[0:partitionIndex])
        inWord = inWord[partitionIndex:]
        partitionIndex = searchForFirstWord(inWord,inDict,debug)
    if(len(inWord)>0):
        wordVec.append(inWord)
        outHashtag.append(isHashtag)
        outEmot.append(isEmot)
    if(len(wordVec)==0):
        wordVec = [origWord]
        outHashtag = isHashtag
    if(debug):
        print(
            "%s was partitioned into %i words: %s \n with hashtag: %s"
            % (origWord,len(wordVec),wordVec,outHashtag)
             )
    assert (len(wordVec) == len(outHashtag) and len(outHashtag) == len(outEmot))
    return(wordVec,outHashtag,outEmot)

### for a single sentence, split the sentence into individual words, and split compacted words that are missing a space ###

this function also adjusts hashtag and emoji indicator vectors to accomadate expanded list of words

**Inputs:** <br>
- **inSent** (string) - entire sentence <br>
- **inDict** (dict) - dictionary of allowed, or 'legal' words <br>
- **inHashtag** (int array) - indicator of which words in the sentence, if any, belong to a hashtag <br>
- **inEmot** (int array) - indicator of which words in the sentence, if any, belong to an emoji description <br>
- **debug** (boolean) - whether debug statements should be printed <br>

**Outputs:** <br>
- **splitSent** (string array) - revised sentence in array format, with each word an element in the array <br>
- **outHashtag** (int array) - revised hashtag indicator array <br>
- **outEmot** (int array) - revised emot indicator array <br>

In [14]:
def screenForCompactedWordsInSentence(inSent,inDict,inHashtag,inEmot,debug=False):
    splitSent, outHashtag, outEmot = ([] for i in range(3))
    wordIndex = 0
    for word in inSent.split(" "):
        if(len(word)>0):
            try:
                currHashtag = inHashtag[wordIndex]
                currEmot = inEmot[wordIndex]
            except Exception as e:
                if(debug):
                    print(str(e))
            if not word in inDict:
                splitWords,hashtags,emotes = checkOneCompactedWord(word,inDict,currHashtag,currEmot,debug)
                splitSent += splitWords
                outHashtag += hashtags 
                outEmot += emotes
            else:
                splitSent.append(word)
                outHashtag.append(currHashtag)
                outEmot.append(currEmot)
            if(word not in string.punctuation and word not in ['...']):
                wordIndex +=1
    if(debug):
        print(
            "sentence after splitting: %s \n hashtag: %s \n emote: %s"
            % (splitSent,outHashtag,outEmot)
        )
    assert(len(splitSent) == len(outHashtag) & len(outHashtag) == len(outEmot))
    return splitSent, outHashtag, outEmot

### for all sentences in the dataset, split the sentences into individual words, and split compacted words that are missing a space ###

**Inputs:** <br>
- **TweetSentences** (string array) - sentences to process <br>
- **inDict** (dict) - dictionary of allowable or 'legal' words <br>
- **inHashtag** (list of integer arrays) - one integer array for each sentence, indicating which words belong to a hashtag <br>
- **inEmot** (list of integer arrays) - one integer array for each sentence, indicating which words belong to an emoticon description <br>
- **debug** (boolean) - whether to print debug statements <br>

**Outputs:**<br>
- **screenedSents** (list of string arrays) - revised version of Tweet sentences, with compacted words partitioned into multiple words.  Each sentence is now an arrays of strings instead of a single string <br>
- **screenedHashtags** (list of integer arrays) - revised version of inHashtags, adjusted for word insertions <br>
- **screenedEmots** (list of integer arrays) - revised version of inEmot, adjusted for word insertions <br>


In [15]:
def screenForCompactedWordsAllTweets(TweetSentences,inDict,inHashtag,inEmot,debug=False):
    screenedSents, screenedHashtags, screenedEmots = ([] for i in range(3))
    screenedHashtags = []
    screenedEmots = []
    for currIndex in range(10000):#(len(TweetSentences)):
        splitSent, updatedHashtag, updatedEmot = screenForCompactedWordsInSentence(
            TweetSentences[currIndex],
            inDict,
            inHashtag[currIndex],
            inEmot[currIndex],
            debug = False
        )
        screenedSents.append(splitSent)
        screenedHashtags.append(updatedHashtag)
        screenedEmots.append(updatedEmot)
    if(debug):
        print(
            "Example screening for compact words: \n %s, %s, %s"
            % (screenedSents[0],screenedHashtags[0],screenedEmots[0])
        )
        print(
            "Number of tweets screened: %i" %len(screenedSents)
        )
    assert(len(screenedSents) == len(screenedHashtags) & len(screenedHashtags) == len(screenedEmots))
    return(screenedSents,screenedHashtags,screenedEmots)

In [98]:
# remove word encodings whose corresponding word isn't in the Twiter dataset
def reduceWordEmbeddings(embeddedWeights,word2indexMap):
    reducedEmbeddingWeights = {}
    for key, value in embeddedWeights.items():
        if key in word2indexMap:
            reducedEmbeddingWeights[key] = embeddedWeights[key]
    return reducedEmbeddingWeights

In [114]:
# replace a single word not found in the dictionary with an unknown token
def tagSingleWord(word,unknownTag,word2index_map):
    stripPunc = string.punctuation.replace("'"," ")
    word = word.translate({ord(x): '' for x in stripPunc})
    if(len(word)>0 and word[0] == "'"):
        word = word[1:]
    if(len(word)>0 and word[-1] == "'"):
        word = word[:-1]
    if(len(word)>0):
        if word not in word2index_map:
            word = unknownTag
        return(word)
    return(None)

In [135]:
# for all sentences, replace all words not found in the dictionary with an unknown token
def substituteAllUnknownTags(inputSentences,unknownTag,word2index_map):
    taggedSentences = []
    for sent in inputSentences:
        screenedSentence = ""  
        for word in sent:
            taggedWord = tagSingleWord(word,unknownTag,word2index_map)
            if not taggedWord == None: screenedSentence += taggedWord + " "
        screenedSentence = screenedSentence[:-1]
        taggedSentences.append(screenedSentence)
    return(taggedSentences)

In [107]:
# map word encoding to corresponding word, and create a lookup dictionary.
def convertVectorsToMatrix(word2embedding_dict,vec_dim):
    embedding_matrix = np.zeros((len(word2embedding_dict)+2,vec_dim))
    wordToIndex = {}
    index = 0
    for word, vector in word2embedding_dict.items():
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
        wordToIndex[word] = index
        index +=1
    return(embedding_matrix,wordToIndex)

In [70]:
# identify unknown words and their corresponding frequency
def countUnknownWords(embeddingWeights,screenedSents):
    unknownWordDict = {}
    for sent in screenedSents:
        for word in sent:
            if word not in embeddingWeights:
                if word in unknownWordDict:
                    unknownWordDict[word] = unknownWordDict[word] + 1
                else:
                    unknownWordDict[word] = 1
    return(unknownWordDict)

In [109]:
# replace unknown words with list of known substitutions
# for example, replace "'ve" with "have"
def substituteUnknownWords(screenedSents,unknownWordDict):
    for sentIndex in range(len(screenedSents)):
        for wordIndex in range(len(screenedSents[sentIndex])):
            if screenedSents[sentIndex][wordIndex] in unknownWordDict:
                screenedSents[sentIndex][wordIndex] = unknownWordDict[screenedSents[sentIndex][wordIndex]]
    return screenedSents

### find the number of words in the longest tweet (by word count) in the database ###
To do this we find the max length of the emoticon indicator vectors, since each vector contains one element for one word in a sentence

**Inputs:** <br>
- **inputEmoticons** (list of integer arrays) - one integer array for each sentence.  Len of array corresponds to number of words in the sentence <br>

**Outputs:** <br>
- **maxSentenceLength** (int) - number of words in the longest tweet (by word count)

In [121]:
def findMaxSentenceLength(inputEmoticons):
    maxSentenceLength = 0
    for emot in inputEmoticons:
        maxSentenceLength = max(len(emot), maxSentenceLength)
    return(maxSentenceLength)

### pad shorter sentences with pad tokens to ensure all sentences have the same vector length ###
this allows data to be stored in matrices, presenting the opportunity to vectorize deep learning and perform minibatch rather than stochastic gradient descent training 

**Inputs:**<br>
- **inputSentences** (list of string arrays) - sentences that need to be padded <br>
- **maxSentenceLength** (int) - number of words that each sentence must contain.  Add pad tokens to the end of shorter sentences to increase their length <br>
- **padToken** (string) - specific word to add to sentences to indicate end of sentence has already been reached <br>

**Outputs:**<br>
- **paddedSents** (list of string arrays) - revised version of inputSentences, with padding added so all sentences have maxSentenceLength number of words <br>
- **sentLengths** (integer array) - number of words in each sentence **before** adding padding <br>

In [137]:
def substituteAllUnknownTags(inputSentences,maxSentenceLength,padToken):
    paddedSents = []
    sentLengths = []
    for sent in inputSentences:
        sentLength = len(sent.split(" "))
        sentLengths.append(sentLength)
        while(sentLength < maxSentenceLength):
            sent = sent + " " + padToken
            sentLength +=1
        paddedSents.append(sent)
    return(paddedSents,sentLengths)

### extend length of an indicator vector so that all indicator vectors are the same as the longest vector ###
This is needed so that all indicator vectors can be combined into a matrix, letting tensorflow vectorize deep learning operations <br>

**Inputs:** <br>
- **inputVectors** (list of integer arrays) - indicator vectors that need to be padded
- **maxSentenceLength** (int) - number of elements each vector must contain.  Add zeros to the end of shorter sentences to increase their length <br>

**Outputs:** <br>
- **paddedVectors** - revised version of inputVectors, with padding added so all vectors have maxSentence length number of elements <br>


In [138]:
def padIndicatorVector(inputVectors,maxSentenceLength):
    paddedVectors = []
    for vector in inputVectors:
        vectorLength = len(vector)
        numVector = np.asarray(list(map(int, vector)),dtype=np.int32).reshape((vectorLength))
        if(maxSentenceLength-vectorLength >0):
            padding = np.zeros((maxSentenceLength-vectorLength,1),dtype=np.int32)
            numVector = np.append(numVector,padding)
            numVector = numVector.reshape(maxSentenceLength)
        paddedVectors.append(numVector)
    return(paddedVectors)

### add padding to all input features to the tensorflow model ###
This ensures all vectors are the same length, greatly increasing tensorflow's ability to vectorize operations and reduce training time.

**Inputs:** <br>
- **inputSentences** (list of string arrays) - sentences that need to be padded <br>
- **inputHashtagInd** (list of integer arrays) - hashtag vectors that need to be padded <br>
- **inputEmoticonInd** (list of integer arrays) - emoticon vectors that need to be padded <br>
- **padToken** (string) - token word to add at end of sentences to make their length the same <br>

**Outputs:** <br>
- **paddedSents** (list of string arrays) - revised version of inputSentences, with padding added so all sentences have maxSentenceLength number of words <br>
- **sentLengths** (int array) - length of each sentence in inputSentences **before** adding padding <br>
- **paddedHashtagInds** - revised version of inputHashtagInd, with padding added so all vectors have maxSentence length number of elements <br>
- **paddedEmoticonInds** - revised version of inputEmoticonIn, with padding added so all vectors have maxSentenceLength number of elements <br>
- **maxSentenceLength** - number of elements each vector contains after padding <br>

In [136]:
def addPadding(inputSentences,inputHashtagInd,inputEmoticonInd,padToken):
    maxSentenceLength = findMaxSentenceLength(inputHashtagInd,inputEmoticonInd)
    paddedSents,sentLengths = padSentences(inputSentences,maxSentenceLength,padToken)
    paddedHashtagInds = padIndicatorVector(inputHashtagInd,maxSentenceLength)
    paddedEmoticonInds = padIndicatorVector(inputEmoticonInd,maxSentenceLength)
    return(paddedSents,sentLengths, paddedHashtagInds,paddedEmoticonInds,maxSentenceLength)

### main function ###

In [122]:
embeddingWeights = loadWordVec(wordVecParams)
tweetDict = loadWordList(twitterCSVParams)
# print(tweetDict.head()) for debug/ validation purposes only
screenedSents,screenedHashtags,screenedEmots = screenForCompactedWordsAllTweets(tweetDict['clean_text'],embeddingWeights,tweetDict['hash_ind'],tweetDict['emot_ind'],True)
index2WordMap, word2IndexMap = generateWordMap(screenedSents)
word2embedding_dict = reduceWordEmbeddings(embeddingWeights,index2WordMap)
# countUnknownWords(word2embedding_dict,screenedSents) # for debug/ validation purposes only
word2embedding_dict['UNK'] = np.random.randn(modelParams['word_vec_dim'])*0.001 # initialize to small but non-zero random weight vector
word2embedding_dict['PAD_TOKEN'] = np.random.randn(modelParams['word_vec_dim'])*0.001 # initialize to small but non-zero random weight vector
embedding_matrix,word2Index = convertVectorsToMatrix(word2embedding_dict,modelParams['word_vec_dim'])
screenedSents = substituteAllUnknownTags(screenedSents,unknownWordSubs)
taggedSentences = SubstituteUnknownTag(screenedSents,"UNK",word2Index)
paddedSent, sentLengths,paddedHashtags,paddedEmoticons,maxSentenceLength = addPadding(taggedSentences, screenedHashtags, screenedEmots, "PAD_TOKEN")